<a href="https://colab.research.google.com/github/amitagarwal2208/Suicide-Intent-Classification/blob/main/LSTM_SuicideIntentIIITD_AmitAgarwal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/AmanuelF/Suicide-Risk-Assessment-using-Reddit/master/anonymized_dataset/500_Reddit_users_posts_labels.csv'
df = pd.read_csv(url)

In [3]:
df = df.dropna()
X=df.drop('Label',axis=1)
Y=df['Label']

In [4]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
messages = X.copy()

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [7]:
messages = []
exclude = ['[',']',',',', ']

for i in range(500):
  l = df['Post'][i].split("'") 
  l2 = []
  for s in l:
    if s not in exclude:
      l2.append(s)

  messages.append(l2)

In [8]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(500):
  l = []
  for text in messages[i]:
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    l.append(review)

  corpus.append(l) 

In [9]:
corpus[19]

['live other say',
 'trigger well cant drink alcohol know much fun want drown drink littl drank today ive issu drink cultur realiti your outsid dont',
 'whole life semi stage depress mix period good time moment want neck commit life prevent continu say ill get past point wont matter fuckhead mate wed famili member import event defer fuck world',
 'reason peopl like differ shit good way',
 'one truli beat depress ive gone fair without major depress stint still get massiv blue day',
 'one day hope get ny hold hug',
 'work peopl doesnt work other ive alway play sport exercis least three time asthenia biggest low session',
 'dog real though',
 'yeah theyr best',
 'ive never anyon',
 'http www southparkstudio com clip beauti sad',
 'went got realli drunk told peopl work day later told go see doctor put lexapro got serious high day',
 'got total fuck one night sex girl fuck amaz almost miss flight home wasnt sure fuck happend got number proceed text proof anyon chick exist besid fuck mind se

In [10]:
corpusConcat = []
for i in range(500):
  s = "" 
  for t in corpus[i]:
    s = s+t 
  corpusConcat.append(s)

In [11]:
import tensorflow_hub as hub

In [12]:
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [13]:
def get_max_length(corpusConcat):
    
    max_length = 0
    for sentence in corpusConcat:
      max_length = max(max_length,len(sentence))
    return max_length

def get_word2vec_enc(corpus):
    encoded_text = []
    for text in corpus:
        tokens = text.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_text.append(word2vec_embedding)
    return encoded_text


def get_padded_encoded_text(encoded_text):

    max_length = get_max_length(encoded_text)
    padded_text_encoding = []
    for enc_text in encoded_text:
        zero_padding_cnt = max_length - enc_text.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_text = np.concatenate((pad, enc_text), axis=0)
        padded_text_encoding.append(enc_text)
    return padded_text_encoding



def label_encode(label):
  
    if label == 'Attempt':
        return [1,0,0,0,0]
    elif label == 'Behavior':
        return [0,1,0,0,0]
    elif label == 'Ideation':
        return [0,0,1,0,0]
    elif label == 'Indicator':
        return [0,0,0,1,0]
    elif label == 'Supportive':
        return [0,0,0,0,1]


In [14]:
encoded_text = get_word2vec_enc(corpusConcat)

In [15]:
padded_encoded_text = get_padded_encoded_text(encoded_text)

In [16]:
encoded_labels = [label_encode(label) for label in Y]

In [17]:
X1 = padded_encoded_text
X1 = np.array(X1)

In [18]:
Y1 = encoded_labels
Y1 = np.array(Y1)

In [19]:
from tensorflow.keras.layers import Dropout

In [20]:
X_train = X1[:375]
X_test = X1[375:]
Y_train = Y1[:375]
Y_test = Y1[375:]

In [21]:
Y1.shape

(500, 5)

In [24]:
model = Sequential()
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_test, Y_test),
          epochs=4, batch_size=12)

Epoch 1/4
32/32 [==============================] - 90s 3s/step - loss: 1.5599 - accuracy: 0.3120 - val_loss: 1.5349 - val_accuracy: 0.3200
Epoch 2/4
32/32 [==============================] - 87s 3s/step - loss: 1.5288 - accuracy: 0.3413 - val_loss: 1.5292 - val_accuracy: 0.3200
Epoch 3/4
32/32 [==============================] - 87s 3s/step - loss: 1.5208 - accuracy: 0.3493 - val_loss: 1.5281 - val_accuracy: 0.3200
Epoch 4/4
32/32 [==============================] - 87s 3s/step - loss: 1.5069 - accuracy: 0.3493 - val_loss: 1.5324 - val_accuracy: 0.3120
